In [ ]:
import zipfile
import os
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import shutil

In [ ]:
with zipfile.ZipFile("../input/dogs-vs-cats-redux-kernels-edition/train.zip","r") as z:
    z.extractall(".")

In [ ]:
data_dir = "./train"

fig = plt.figure(figsize=(16,8))
i = 1
cols = 3
rows = 3
for img in os.listdir(data_dir):
    if(i == 10):
        break
    img_arr = cv2.imread(os.path.join(data_dir,img))
    fig.add_subplot(rows,cols,i)
    plt.imshow(img_arr)
    plt.title(img.split(sep=".")[0])
    i+=1
plt.show()


In [ ]:
new_shape = 224
new_arr = cv2.resize(img_arr, (new_shape,new_shape))
plt.imshow(new_arr)
plt.show()
new_arr.shape

In [ ]:
os.mkdir('data')
os.mkdir('./data/dogs')
os.mkdir('./data/cats')
os.mkdir('val')
os.mkdir('./val/dogs')
os.mkdir('./val/cats')


path2d = "./data/dogs"
path2c = "./data/cats"
path2dval = "./val/dogs"
path2cval = "./val/cats"

In [ ]:
i = 0;
for img in os.listdir(data_dir):
    if(img.split(".")[0] == 'cat'):
        if(i<20000):
            shutil.move(os.path.join(data_dir,img), os.path.join(path2c,img))
        else:
            shutil.move(os.path.join(data_dir,img), os.path.join(path2cval,img))
            
    else:
        if(i<20000):
            shutil.move(os.path.join(data_dir,img), os.path.join(path2d,img))
        else:
            shutil.move(os.path.join(data_dir,img), os.path.join(path2dval,img))
    i+=1

In [ ]:
cats_dir = [img.split(".")[0] for img in os.listdir(path2c)]
dogs_dir = [img.split(".")[0] for img in os.listdir(path2d)]
cats_dir_val = [img.split(".")[0] for img in os.listdir(path2cval)]
dogs_dir_val = [img.split(".")[0] for img in os.listdir(path2dval)]
print("cats ",cats_dir[:5]," ",len(cats_dir))
print("dogs ",dogs_dir[:5]," ",len(dogs_dir))
print("cats val",cats_dir_val[:5]," ",len(cats_dir_val))
print("dogs val",dogs_dir_val[:5]," ",len(dogs_dir_val))

In [ ]:
dataset = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function = tf.keras.applications.vgg16.preprocess_input,
    rescale=1.0/255.0,
    horizontal_flip=True,
).flow_from_directory(
    './data', target_size=(new_shape, new_shape), color_mode='rgb', classes=['cats','dogs'],
    class_mode='binary', batch_size=20, shuffle=True)

valset = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function = tf.keras.applications.vgg16.preprocess_input,
    rescale=1.0/255.0,
    horizontal_flip=True,
).flow_from_directory(
    './val', target_size=(new_shape, new_shape), color_mode='rgb', classes=['cats','dogs'],
    class_mode='binary', batch_size=20, shuffle=True)


dataset.class_indices
valset.class_indices


In [ ]:
img, label = next(dataset)

fig = plt.figure(figsize=(20,10))
rows = 2
cols = 10
i = 1

for j in range(20):
    fig.add_subplot(rows,cols,i)
    plt.imshow(img[j])
    plt.title(label[j])
    i+=1
plt.show()

In [ ]:
img, label = next(valset)

fig = plt.figure(figsize=(20,10))
rows = 2
cols = 10
i = 1

for j in range(20):
    fig.add_subplot(rows,cols,i)
    plt.imshow(img[j])
    plt.title(label[j])
    i+=1
plt.show()

In [ ]:
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D
from tensorflow.keras import activations

model = tf.keras.models.Sequential()

model.add(Conv2D(64, (3,3), activation="relu" ,input_shape=(new_shape,new_shape,3),padding='same'))
model.add(Conv2D(64, (3,3), activation="relu",padding='same'))
model.add(MaxPool2D(pool_size=(2,2),strides=2))

model.add(Conv2D(128, (3,3), activation="relu",padding='same'))
model.add(Conv2D(128, (3,3), activation="relu",padding='same'))
model.add(MaxPool2D(pool_size=(2,2),strides=2))

model.add(Conv2D(256, (3,3), activation="relu",padding='same'))
model.add(Conv2D(256, (3,3), activation="relu",padding='same'))
model.add(Conv2D(256, (3,3), activation="relu",padding='same'))
model.add(MaxPool2D(pool_size=(2,2),strides=2))

model.add(Conv2D(512, (3,3), activation="relu",padding='same'))
model.add(Conv2D(512, (3,3), activation="relu",padding='same'))
model.add(Conv2D(512, (3,3), activation="relu",padding='same'))
model.add(MaxPool2D(pool_size=(2,2),strides=2))

model.add(Conv2D(512, (3,3), activation="relu",padding='same'))
model.add(Conv2D(512, (3,3), activation="relu",padding='same'))
model.add(Conv2D(512, (3,3), activation="relu",padding='same'))
model.add(MaxPool2D(pool_size=(2,2),strides=2))


model.add(Flatten())
model.add(Dense(4096, activation="relu"))
model.add(Dense(4096, activation="relu"))
model.add(Dense(1, activation="sigmoid"))

model.compile(optimizer=tf.keras.optimizers.RMSprop(), #learning_rate=0.001 is base
             loss=tf.keras.losses.BinaryCrossentropy(),
             metrics=['accuracy', 'Precision', 'Recall'])

In [ ]:
model.summary()

In [ ]:
history = model.fit(dataset, epochs=40, validation_data=valset, validation_steps=20)

In [ ]:
pd.DataFrame(history.history).plot()
pd.DataFrame(history.history['loss']).plot()
pd.DataFrame(history.history['accuracy']).plot()
pd.DataFrame(history.history['precision']).plot()
pd.DataFrame(history.history['recall']).plot()

In [ ]:
os.mkdir('./test')
with zipfile.ZipFile("../input/dogs-vs-cats-redux-kernels-edition/test.zip","r") as z:
    z.extractall("./test")

In [ ]:
testset = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function = tf.keras.applications.vgg16.preprocess_input,
    rescale=1.0/255.0,
).flow_from_directory(
    './test', target_size=(new_shape, new_shape), color_mode='rgb',
    class_mode=None, batch_size=20, shuffle=True)

In [ ]:
predictions = model.predict(testset)

In [ ]:
d = pd.DataFrame(np.round(predictions))
d.index +=1
d.columns = ['label']
d.index.names = ['id']
d.to_csv('submission.csv')